# Barotropic Stream Function

- BSF minimum in the gyre region [64.9S,337.8E]

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from BSF import calculate_BSF, DP_transport, WG_transport
from grid import find_array_idx
from paths import file_ex_ocn_ctrl, path_results, path_samoc
from regions import Drake_Passage, DP_North
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_DZ, xr_DYU

In [ ]:
%%time
# 20 sec per file
# run = 'ctrl'
run = 'rcp'
for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='yrly', name='UVEL_VVEL')):
    # ca. 30 s per year
    print(i, y, s)
    ds = xr.open_dataset(s, decode_times=False)
    
    BSF = calculate_BSF(ds)
    ds.close()
#     del BSF.encoding["contiguous"]
#     if i==0:
#         BSF_out = BSF.copy()
#         BSF_out = xr.concat([BSF_out, BSF], dim='time')
#     if y==201: break
    BSF.to_netcdf(f'{path_samoc}/BSF/BSF_{run}_{y}.nc')
    BSF.close()
#     if y==101: break
combined = xr.open_mfdataset(f'{path_samoc}/BSF/BSF_{run}_*.nc',
                             concat_dim='time',
                             autoclose=True,
                             coords='minimal')
combined.to_netcdf(f'{path_samoc}/BSF/BSF_{run}.nc')

In [ ]:
BSF

In [ ]:
BSF_ctrl = xr.open_dataset(f'{path_samoc}/BSF/BSF_{run}.nc', decode_times=False)

In [ ]:
BSF_ctrl

In [ ]:
run = 'ctrl'
xr.set_options(file_cache_maxsize=8)
combined = xr.open_mfdataset(f'{path_samoc}/BSF/BSF_{run}_*.nc',
                             concat_dim='time',
                             autoclose=True,
                             coords='minimal')
combined.to_netcdf(f'{path_samoc}/BSF/BSF_{run}.nc')

In [ ]:
BSF_ctrl = xr.open_dataarray(f'{path_samoc}/BSF/BSF_ctrl.nc', decode_times=False)
BSF_rcp  = xr.open_dataarray(f'{path_samoc}/BSF/BSF_rcp.nc' , decode_times=False)

In [ ]:
print(len(BSF_ctrl))
print(len(BSF_rcp))


In [ ]:
BSF_rcp

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.plot(DP_transport(BSF_ctrl).values/1e6, lw=2, label='CTRL')
plt.plot(DP_transport(BSF_rcp ).values /1e6, lw=2, label='RCP')

plt.xlabel('time [years]', fontsize=16)
plt.ylabel('Drake Passage transport [Sv]', fontsize=16)
plt.legend(fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/BSF/BSF_DP_timeseries')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.plot(WG_transport(BSF_ctrl).values/1e6, lw=2, label='CTRL')
plt.plot(WG_transport(BSF_rcp ).values /1e6, lw=2, label='RCP')

plt.xlabel('time [years]', fontsize=16)
plt.ylabel('Weddell Gyre transport [Sv]', fontsize=16)
plt.legend(fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/BSF/BSF_WG_timeseries')